In [2]:
#JSS

import numpy as np
from pathlib import Path
import cv2
import torch
from hfnet.models import get_model
import tensorflow as tf
import os

from hloc.utils.base_model import dynamic_load
from hloc import extractors
from hloc.utils.tools import map_tensor
# import Mask_RCNN_class
feature_config ={'model': {'max_keypoints': 4096, 'name': 'superpoint', 'nms_radius': 3},'preprocessing': {'grayscale': True, 'resize_max': 960}}

global_config = {'data': {'name': 'aachen', 'load_db': False, 'load_queries': True, 'resize_max': 960}, 'model': {'name': 'netvlad_original', 'local_descriptor_layer': 'conv3_3', 'image_channels': 1}, 'weights': 'vd16_pitts30k_conv5_3_vlad_preL2_intra_white/vd16_pitts30k_conv5_3_vlad_preL2_intra_white'}
path_for_weights_netVLAD = "/home/Hierarchical-Localization/weights/vd16_pitts30k_conv5_3_vlad_preL2_intra_white/vd16_pitts30k_conv5_3_vlad_preL2_intra_white"
# os.environ["CUDA_VISIBLE_DEVICES"] = '0,1,4'

%matplotlib inline 


In [9]:
class Query:
    feature_config ={'model': {'max_keypoints': 4096, 'name': 'superpoint', 'nms_radius': 3},'preprocessing': {'grayscale': True, 'resize_max': 960}}
    
    
    def __init__(self):
        self.load_superpoint()
#         self.load_global_descriptor()
#         self._mask_rcnn = Mask_RCNN_class.Model_RCNN()
        
    def new_image(self,image, use_Mask_RCNN=False):
        self.image = image ## send in BRG instead of RGB 
        
        if (use_Mask_RCNN):
            self.masking()
        self.processed()
        self.key_point_and_descriptor()
        
          
    def masking(self):
        self.image = cv2.cvtColor(self.image, cv2.COLOR_BGR2RGB)
        self.image =self._mask_rcnn .rcnn_results_func(self.image)
        return self.image

    def processed(self):
        if feature_config["preprocessing"]["grayscale"]:
            image_processed = cv2.cvtColor(self.image,cv2.COLOR_RGB2GRAY)
        if image_processed is None:
            raise ValueError(f'Cannot read image {str(path)}.')
        image_processed = image_processed.astype(np.float32)
        size = image_processed.shape[:2][::-1]
        w, h = size

        if feature_config["preprocessing"]["resize_max"] and max(w, h) > feature_config["preprocessing"]["resize_max"]:
            scale = feature_config["preprocessing"]["resize_max"]  / max(h, w)
            h_new, w_new = int(round(h*scale)), int(round(w*scale))
            image = cv2.resize(
                image_processed, (w_new, h_new), interpolation=cv2.INTER_LINEAR)

        if feature_config["preprocessing"]["grayscale"]:
            image_processed = image_processed[None]
        else:
            image_processed = image_processed.transpose((2, 0, 1))  # HxWxC to CxHxW
        image_processed = image_processed / 255.

        self.data = {
            'image': (torch.from_numpy(image_processed)).unsqueeze(0),
            'original_size': torch.from_numpy(np.array([size])),
        }
        return self.data
    
    @torch.no_grad()
    def load_superpoint(self):
        self.device_sp = 'cuda' if torch.cuda.is_available() else 'cpu'
#         with torch.cuda.device(0):
        print("Device for superpoint", self.device_sp)
        Model_sp = dynamic_load(extractors, feature_config['model']['name'])
        self.model_sp = Model_sp(feature_config['model']).eval().to(self.device_sp)

    def key_point_and_descriptor(self):
        pred_sp = self.model_sp(map_tensor(self.data, lambda x: x.to(self.device_sp)))
        pred_sp = {k: v[0].cpu().numpy() for k, v in pred_sp.items()}
        pred_sp['image_size'] = original_size = self.data['original_size'][0].numpy()
        if 'keypoints' in pred_sp:
            size = np.array(self.data['image'].shape[-2:][::-1])
            scales = (original_size / size).astype(np.float32)
            pred_sp['keypoints'] = (pred_sp['keypoints'] + .5) * scales[None] - .5
#         if as_half:
#             for k in pred:
#                 dt = pred[k].dtype
#                 if (dt == np.float32) and (dt != np.float16):
#                     pred[k] = pred[k].astype(np.float16)
# #         name = data['name'][0] ## name of the file
        pred_sp["name"] = "query"
        

        logging.info('Finished exporting features.')
        self.pred_copy = pred_sp 
        del pred ## do not know why they are doing this 
        return self.pred_copy
    def load_global_descriptor(self):
        #with tf.device('/job:localhost/replica:0/task:0/device:XLA_GPU:2'):
        self.net = get_model(global_config['model']['name'])(data_shape={'image': [None, None, None, 3]},**global_config['model'])
        checkpoint_path = Path(path_for_weights_netVLAD)
        self.net.load(str(checkpoint_path))
        

        
        
        


In [10]:
q = Query()

Device for superpoint cuda
Loaded SuperPoint model


In [11]:
image_array_for_feature_detector = cv2.imread("/home/Hierarchical-Localization/datasets/Images_query_folder/iPhoSilWithWv4_1598593698_648-02165.png")

q.new_image(image_array_for_feature_detector)
# i = q.masking()

RuntimeError: Can't call numpy() on Tensor that requires grad. Use tensor.detach().numpy() instead.

In [ ]:
import matplotlib.pyplot as plt
import IPython.display
_, ax = plt.subplots(1, figsize=(16,16))
ax.imshow(i)

In [ ]:
    def load_global_descriptor(self):
        with tf.device('/device:GPU:2'):
            self.net = get_model(global_config['model']['name'])(data_shape={'image': [None, None, None, 3]},**global_config['model'])
            checkpoint_path = Path(path_for_weights_netVLAD)
            self.net.load(str(checkpoint_path))

In [ ]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))